<a href="https://colab.research.google.com/github/zhlu97/pn_pc_cls/blob/master/EasyPointNet_Plot_DuringTraining_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Easiest PointNet Possible

## Fetch ModelNet40 and Decompress

In [ ]:
!wget https://shapenet.cs.stanford.edu/media/modelnet40_normal_resampled.zip

--2020-04-22 18:58:10--  https://shapenet.cs.stanford.edu/media/modelnet40_normal_resampled.zip
Resolving shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)... 171.67.77.19
Connecting to shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)|171.67.77.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1705117335 (1.6G) [application/zip]
Saving to: ‘modelnet40_normal_resampled.zip.2’

modelnet40_normal_r 100%[===================>]   1.59G  2.96MB/s    in 9m 12s  

2020-04-22 19:07:22 (2.95 MB/s) - ‘modelnet40_normal_resampled.zip.2’ saved [1705117335/1705117335]



In [ ]:
%%capture
!unzip modelnet40_normal_resampled.zip

## Import Libraries

In [ ]:
import numpy as np
import warnings
import os
from torch.utils.data import Dataset
warnings.filterwarnings('ignore')

import torch
import logging
# from tqdm import tqdm
from tqdm.notebook import tqdm
import sys
import importlib

import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable

import torch.nn.functional as F
import matplotlib.pyplot as plt
from numpy import genfromtxt

## Define hyperparameters

In [ ]:
batch_size = 60
sample_point_count = 1024
num_classes = 40
num_epoch = 200
learning_rate = 0.001
# rootPath = '/content/modelnet40_normal_resampled/'
rootPath = '/home/isuzu/modelnet40_normal_resampled/'

## Dataloader

### To use Pytorch's built-in dataloader, must implement subclass and override all three functions below.

In [ ]:
class ModelNetResampledLoader(Dataset):
  def __init__(self, rootPath, mode):
        self.categoryNames = [line.rstrip() for line in open(os.path.join(rootPath, 'modelnet40_shape_names.txt'))]
        self.classes = dict(zip(self.categoryNames, range(len(self.categoryNames))))

        self.all_IDs = {}
        self.all_IDs['train'] = [line.rstrip() for line in open(os.path.join(rootPath, 'modelnet40_train.txt'))]
        self.all_IDs['test']  = [line.rstrip() for line in open(os.path.join(rootPath, 'modelnet40_test.txt'))]   

        shape_names = ['_'.join(x.split('_')[0:-1]) for x in self.all_IDs[mode]]
        self.datapath = [(shape_names[i], os.path.join(rootPath, shape_names[i], self.all_IDs[mode][i]) + '.txt') for i
                         in range(len(self.all_IDs[mode]))]        

  def __len__(self):
        'Denotes the total number of samples'
        'num of .txt'
        return len(self.datapath)

  def __getitem__(self, index):
        selectedDatapaths = self.datapath[index]
        selectedClasses   = self.classes[selectedDatapaths[0]]
        selectedClasses   = np.array([selectedClasses]).astype(np.int32)

        pointSet = np.loadtxt(selectedDatapaths[1], delimiter=',').astype(np.float32)
        pointSet = pointSet[0:sample_point_count,:] #Only choosing first 1024? -- Points have no ordering. We can already see shape with first 1024 pts
        pointSet = pointSet[:, 0:3]
        plt.plot(pointSet)

        return pointSet, selectedClasses

## Simpliest PointNet Model

In [ ]:
class STNkd(torch.nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()

        self.expand = torch.nn.Sequential(
            torch.nn.Conv1d(k, 64, 1),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),
            torch.nn.Conv1d(64, 128, 1),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Conv1d(128, 1024, 1),
            torch.nn.BatchNorm1d(1024),
            torch.nn.ReLU()
        )

        self.shrink = torch.nn.Sequential(
            torch.nn.Linear(1024, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 256),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, k * k),
        )

        self.k = k     

    def forward(self, x):
        batchsize = x.size()[0]
        x = self.expand(x)
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        x = self.shrink(x)

        init_id = Variable(torch.eye(self.k)).view(1, self.k * self.k).repeat(batchsize, 1).cuda()
        x += init_id
        x = x.view(-1, self.k, self.k)

        return x

class PointNetEncoder(torch.nn.Module):
    def __init__(self, channel=3):
        super(PointNetEncoder, self).__init__()

        self.tNet3 = STNkd(channel)
        self.featureExpansion_3_64 = torch.nn.Sequential(
            torch.nn.Conv1d(channel, 64, 1),
            torch.nn.BatchNorm1d(64)
        )
        self.tNet64 = STNkd(64)
        self.featureExpansion_64_128_1024 = torch.nn.Sequential(
            torch.nn.Conv1d(64, 64, 1),
            torch.nn.BatchNorm1d(64),
            torch.nn.Conv1d(64, 128, 1),
            torch.nn.BatchNorm1d(128),
            torch.nn.Conv1d(128, 1024, 1),
            torch.nn.BatchNorm1d(1024)
        )

    def forward(self, x):
        B, D, N = x.size()

        inputTransform = self.tNet3(x)
        
        x = x.transpose(2, 1)
        x = torch.bmm(x, inputTransform)
        x = x.transpose(2, 1)

        x = self.featureExpansion_3_64(x)
        featTranform = self.tNet64(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, featTranform)
        x = x.transpose(2, 1)

        pointfeat = x
        x = self.featureExpansion_64_128_1024(x)
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        return x, inputTransform, featTranform

class Model(torch.nn.Module):
    def __init__(self, channel = 3):
        super(Model, self).__init__()

        self.pointNetEncoder = PointNetEncoder(channel=channel)

        self.pointNetDecoder = torch.nn.Sequential(
            torch.nn.Linear(1024, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 256),
            torch.nn.Dropout(p=0.4),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, num_classes),
            torch.nn.Softmax(dim = 1)
        )

    def forward(self, x):
        x, inputTransform, featTranform = self.pointNetEncoder(x)
        x = self.pointNetDecoder(x)

        return x

class Loss(torch.nn.Module):
    def __init__(self):
        super(Loss, self).__init__()

    def forward(self, pred, target):
        loss = torch.nn.functional.nll_loss(pred, target)
        return loss

## Training

In [ ]:
# Get data loader
trainData = ModelNetResampledLoader(rootPath=rootPath, mode='train')
testData  = ModelNetResampledLoader(rootPath=rootPath, mode='test')
trainDataLoader = torch.utils.data.DataLoader(trainData, batch_size=batch_size, shuffle=True, num_workers=4)
testDataLoader  = torch.utils.data.DataLoader(testData, batch_size=batch_size, shuffle=False, num_workers=4)

# Get model
model = Model().cuda()
criterion = Loss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# # Print model's state_dict
# print("Model's state_dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# # Print optimizer's state_dict
# print("Optimizer's state_dict:")
# for var_name in optimizer.state_dict():
#     print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
print('Start training... Batch size： %d' % batch_size)
for epoch in range(0,num_epoch):
    correctRates = []
    for batch_id, data in tqdm(enumerate(trainDataLoader, 0), total=len(trainDataLoader)):
        points, target = data

        points = points.data.numpy()
        points = torch.Tensor(points)
        target = target[:, 0]

        points = points.transpose(2, 1)
        points, target = points.cuda(), target.cuda()
        optimizer.zero_grad()

        model = model.train()
        pred, _ = model(points)

        loss = criterion(pred, target.long())
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(target.long().data).cpu().sum()
        correctRates.append(correct.item() / float(points.size()[0]))
        loss.backward()
        optimizer.step()

    ###plot###
    # pointsP = points.cpu().numpy()
    # x, y, z = pointsP[0, 0, :], pointsP[0, 1, :], pointsP[0, 2, :]
    # ax = plt.subplot(111, projection='3d')  # 3d plot
    # ax.scatter(x[:], y[:], z[:], c='y')
    # ax.set_zlabel('Z') 
    # ax.set_ylabel('Y')
    # ax.set_xlabel('X')
    
    # max_indices = max_indices.cpu().numpy()
    # x, y, z = pointsP[0, 0, max_indices[0]], pointsP[0, 1, max_indices[0]], pointsP[0, 2,max_indices[0]]
    # # x, y, z = tmp[0, max_indices[0], 0], tmp[0, max_indices[0], 1], tmp[0, max_indices[0], 2]
    # # x, y, z = tmp[0, 0, max_indices[0]], tmp[0, 1, max_indices[0]], tmp[0, 2,max_indices[0]]
    # # x = x.cpu().numpy
    # # y = y.cpu().numpy
    # # z = z.cpu().numpy
    # # print(max_indices)
    # print(x)
    # print(y)
    # ax.scatter(x[:], y[:], z[:], c='r')
    # plt.show()
    if epoch % 10 == 0:
      testPoints = genfromtxt('/home/isuzu/modelnet40_normal_resampled/chair/chair_0001.txt',delimiter=',')
      testPoints = testPoints[0:1024, 0:3]
      testPoints = torch.tensor(testPoints).cuda()

      testPoints = torch.transpose(testPoints, 0, 1)
      testPoints = testPoints.view(-1, 3, 1024)
      testPoints = testPoints.float()
      model.eval()
      predTest, max_indices = model(testPoints)
      # print(testPoints)
      testPoints = testPoints.cpu().numpy()
      # print(testPoints.shape)
      x, y, z = testPoints[:, 0, :], testPoints[:, 1, :], testPoints[:, 2, :]
      ax = plt.subplot(111, projection='3d')  # 3d plot
      ax.scatter(x[:], y[:], z[:], c='y')

      max_indices = max_indices.cpu().numpy()
      x, y, z = testPoints[0, 0, max_indices[0]], testPoints[0, 1, max_indices[0]], testPoints[0, 2,max_indices[0]]
      ax.scatter(x[:], y[:], z[:], c='r')
      plt.show()
    ##########

    trainEpochAccuracy = np.mean(correctRates)
    print('Train Epoch %d Accuracy: %f' %(epoch, trainEpochAccuracy))

torch.save(model, 'model.pytorch')

Start training... Batch size： 60


## Testing

In [ ]:
# Model class must be defined somewhere
model = torch.load('model.pytorch')

model.eval()
mean_correct = []
class_acc = np.zeros((num_classes,3))

for batch_id, data in tqdm(enumerate(testDataLoader), total=len(testDataLoader)):
    points, target = data
    target = target[:, 0]
    points = points.transpose(2, 1)
    points, target = points.cuda(), target.cuda()
    classifier = model.eval()
    pred, summer = classifier(points)
    pred_choice = pred.data.max(1)[1].type(torch.cuda.IntTensor)
    equalCount = (pred_choice == target).cpu().sum()
    mean_correct.append(equalCount.item() / float(points.size()[0]))
instance_acc = np.mean(mean_correct)
print('Test Instance Accuracy: %f' % instance_acc)


Test Instance Accuracy: 0.690079


In [ ]:
# p = torch.tensor([[[2, 3, 5], [6, 1 ,0],[6, 1 ,10],[100,0,-1]], 
#                   [[6, 1 ,0], [2, 3, 5],[6, 1 ,10], [100,0,-2]], 
#                   [[6, 1 ,0], [2, 3, 5],[6, 1 ,10], [100,0,-2]], 
#                   [[6, 1 ,0], [2, 3, 5],[6, 1 ,10], [100,0,-2]], 
#                   [[6, 1 ,0], [2, 3, 5],[6, 1 ,10], [100,0,-2]]]).cuda()
# x, max_idx = torch.max(p, 1, keepdim=True)
# max_idx = max_idx.view(-1, 3)
# # print('type(p): ', type(p))
# # print('p.shape: ', p.shape)
# print(max_idx)
# y = p[1,max_idx[1],:]
# print(y)

In [ ]:
# Model class must be defined somewhere
model = torch.load('model.pytorch')
model.eval()


testPoints = genfromtxt('/content/modelnet40_normal_resampled/airplane/airplane_0001.txt',delimiter=',')
testPoints = testPoints[0:1024, 0:3]
testPoints = torch.tensor(testPoints).cuda()

testPoints = torch.transpose(testPoints, 0, 1)
testPoints = testPoints.view(-1, 3, 1024)
testPoints = testPoints.float()

pred, max_indices = model(testPoints)
print(testPoints)
testPoints = testPoints.cpu().numpy()
print(testPoints.shape)
x, y, z = testPoints[:, 0, :], testPoints[:, 1, :], testPoints[:, 2, :]
ax = plt.subplot(111, projection='3d')  # 3d plot
ax.scatter(x[:], y[:], z[:], c='y')

# y = testPoints[0, max_indices[0], :]
# ax = plt.subplot(111, projection='3d')
# ax.scatter(x[:], y[:], z[:], c='r')
max_indices = max_indices.cpu().numpy()
print(max_indices.)
x, y, z = testPoints[0, 0, max_indices[0]], testPoints[0, 1, max_indices[0]], testPoints[0, 2,max_indices[0]]
# ax = plt.subplot(111, projection='3d')
ax.scatter(x[:], y[:], z[:], c='r')
# plt.show()

# points, target = data
# target = target[:, 0]
# points = points.transpose(2, 1)
# points, target = points.cuda(), target.cuda()
# classifier = model.eval()
# pred = classifier(points)
# pred_choice = pred.data.max(1)[1].type(torch.cuda.IntTensor)
# equalCount = (pred_choice == target).cpu().sum()
# mean_correct.append(equalCount.item() / float(points.size()[0]))